In [1]:
import requests
import zipfile
import io

# Download the zip file
url = "http://thetis.image.ece.ntua.gr/databases/VIDEO_RGB.zip"
response = requests.get(url)
with zipfile.ZipFile(io.BytesIO(response.content), 'r') as zip_ref:
    zip_ref.extractall("/tennis")

In [2]:
import os


folder_paths = [
    "/tennis/VIDEO_RGB/flat_service",
    "/tennis/VIDEO_RGB/forehand_volley",
    "/tennis/VIDEO_RGB/backhand_slice"
]

# Function to rename files sequentially within a folder
def rename_files(folder_path):
    # Get the list of files in the folder
    files = os.listdir(folder_path)


    # Iterate through the files and rename them sequentially
    for i, filename in enumerate(files, start=1):
        # Construct the new filename with the desired numbering format
        new_filename = f"{i}.avi"

        # Construct the full paths for the old and new filenames
        old_filepath = os.path.join(folder_path, filename)
        new_filepath = os.path.join(folder_path, new_filename)

        # Rename the file
        os.rename(old_filepath, new_filepath)


# Rename files in each folder
for folder_path in folder_paths:
    rename_files(folder_path)
    print("Done\n")


Done

Done

Done



In [3]:
import numpy as np

# Define the total number of items per class
total_items_per_class = 165

# Define the number of classes
num_classes = 3

# Calculate the total number of items
total_items = total_items_per_class * num_classes

# Generate indices for all items
all_indices = [(item, class_label) for class_label in range(num_classes)
               for item in range(1, total_items_per_class + 1)]

# Shuffle the indices
np.random.shuffle(all_indices)

# Calculate split sizes
total_samples_per_class = total_items_per_class
train_size_per_class = int(0.8 * total_samples_per_class)
val_size_per_class = int(0.1 * total_samples_per_class)
test_size_per_class = total_samples_per_class - train_size_per_class - val_size_per_class

# Initialize split indices
train_idx = []
val_idx = []
test_idx = []

# Split indices for each class
for class_label in range(num_classes):
    class_indices = [(item, label) for item, label in all_indices if label == class_label]
    np.random.shuffle(class_indices)

    train_idx.extend(class_indices[:train_size_per_class])
    val_idx.extend(class_indices[train_size_per_class:train_size_per_class+val_size_per_class])
    test_idx.extend(class_indices[train_size_per_class+val_size_per_class:total_samples_per_class])

print("Train Index:", train_idx)
print("Val Index:", val_idx)
print("Test Index:", test_idx)

Train Index: [(80, 0), (66, 0), (145, 0), (147, 0), (75, 0), (131, 0), (29, 0), (116, 0), (113, 0), (61, 0), (71, 0), (144, 0), (42, 0), (93, 0), (76, 0), (138, 0), (83, 0), (56, 0), (150, 0), (51, 0), (45, 0), (37, 0), (97, 0), (16, 0), (62, 0), (90, 0), (100, 0), (40, 0), (5, 0), (25, 0), (60, 0), (32, 0), (164, 0), (92, 0), (157, 0), (70, 0), (102, 0), (85, 0), (136, 0), (148, 0), (130, 0), (94, 0), (3, 0), (1, 0), (161, 0), (159, 0), (72, 0), (24, 0), (11, 0), (35, 0), (112, 0), (15, 0), (28, 0), (41, 0), (9, 0), (58, 0), (30, 0), (105, 0), (103, 0), (47, 0), (88, 0), (49, 0), (34, 0), (57, 0), (50, 0), (63, 0), (89, 0), (165, 0), (7, 0), (153, 0), (124, 0), (141, 0), (140, 0), (99, 0), (107, 0), (118, 0), (74, 0), (110, 0), (10, 0), (22, 0), (77, 0), (23, 0), (135, 0), (79, 0), (134, 0), (69, 0), (44, 0), (163, 0), (13, 0), (142, 0), (84, 0), (156, 0), (86, 0), (27, 0), (155, 0), (109, 0), (59, 0), (68, 0), (38, 0), (152, 0), (36, 0), (104, 0), (151, 0), (111, 0), (54, 0), (26, 0)

In [4]:
import os
import numpy as np
import cv2


def count_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    return total_frames



def get_video_tensor(element_tuple, num_frames=30):
    idx, class_label = element_tuple
    class_folder = {0: "/tennis/VIDEO_RGB/flat_service",
                    1: "/tennis/VIDEO_RGB/forehand_volley",
                    2: "/tennis/VIDEO_RGB/backhand_slice"}


    # Load video using OpenCV
    video_path = os.path.join(class_folder[class_label], f"{idx}.avi")
    toatal_frames = count_frames(video_path)
    per_frame = toatal_frames // num_frames
    cap = cv2.VideoCapture(video_path)

    frames = []
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Extract frames with even time distances
        if frame_count % per_frame == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
            frame_rgb = cv2.resize(frame_rgb, (299, 299), interpolation = cv2.INTER_AREA)
            frames.append(frame_rgb)

        # Break if the desired number of frames is reached
        if len(frames) == num_frames:
            break

        frame_count += 1

    cap.release()

    frames = np.array(frames)

    return frames


In [5]:
from keras.models import Model, Sequential, load_model
from keras.layers import Input, LSTM, Dense, TimeDistributed, Lambda, Dropout, Concatenate
from keras import backend as K
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalMaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [6]:
# get the feature outputs of second-to-last layer (final FC layer)


base_model = Xception(weights='imagenet', include_top=True)

# get the feature outputs of second-to-last layer (final FC layer)
outputs = base_model.get_layer('avg_pool').output

cnn_model = Model(inputs=base_model.input, outputs=outputs)

91884032/91884032 [==============================] - 1s 0us/step


In [7]:
from tensorflow.keras.applications.xception import preprocess_input
seq = []
for data in train_idx:
  vid = get_video_tensor(data, 16)
  _, label = data
  for frame in vid:
    frame = np.expand_dims(frame, axis=0)
    frame = preprocess_input(frame)
    features = cnn_model.predict(frame)
    seq.append(features[0])

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 44ms/step


In [ ]:
path = os.path.join('/', 'train_features.npy')
np.save(path, seq)

In [8]:
seq_val = []
for data in val_idx:
  vid = get_video_tensor(data, 16)
  _, label = data
  for frame in vid:
    frame = np.expand_dims(frame, axis=0)
    frame = preprocess_input(frame)
    features = cnn_model.predict(frame)
    seq_val.append(features[0])

1/1 [==============================] - 0s 25ms/step


In [ ]:
path = os.path.join('/', 'val_features.npy')
np.save(path, seq_val)

In [9]:
seq_test = []
for data in test_idx:
  vid = get_video_tensor(data, 16)
  _, label = data
  for frame in vid:
    frame = np.expand_dims(frame, axis=0)
    frame = preprocess_input(frame)
    features = cnn_model.predict(frame)
    seq_test.append(features[0])

1/1 [==============================] - 0s 22ms/step


In [ ]:
path = os.path.join('/', 'test_features.npy')
np.save(path, seq_test)

In [10]:
X_train = np.array(seq)
X_train = X_train.reshape(len(train_idx), 16, 2048)
X_test = np.array(seq_test)
X_test = X_test.reshape(len(test_idx), 16, 2048)

y_train = np.array([y for (x, y) in train_idx])
y_train = np.eye(num_classes)[y_train]

y_test = np.array([y for (x, y) in test_idx])
y_test = np.eye(num_classes)[y_test]

X_val = np.array(seq_val)
X_val = X_val.reshape(len(val_idx), 16, 2048)

y_val = np.array([y for (x, y) in val_idx])
y_val = np.eye(num_classes)[y_val]

In [31]:
class LSTM_model():

    def __init__(self, num_features=2048, hidden_units=256, dense_units=256, reg=1e-1, dropout_rate=1e-1, seq_length=16, num_classes=3):
            # hidden_units: dimension of cell
            # dense_units: number of neurons in fully connected layer above LSTM
            # reg: regularization for LSTM and dense layer
            # - currently adding L2 regularization for RNN connections, and for inputs to dense layer

            model = Sequential()

            # return_sequences flag sets whether hidden state returned for each time step
            # NOTE: set return_sequences=True if using TimeDistributed, else False


            # LSTM layer (dropout)
            model.add(Dropout(dropout_rate, input_shape=(seq_length, num_features)))  # input to LSTM
            model.add(LSTM(hidden_units, return_sequences=True))

            # --- AVERAGE LSTM OUTPUTS --- #

            # dropout between LSTM and softmax
            model.add(TimeDistributed(Dropout(dropout_rate)))

            # commenting out additional FC layer for now
            model.add(TimeDistributed(Dense(dense_units)))

            # apply softmax
            model.add(TimeDistributed(Dense(num_classes, activation="softmax")))

            # average outputs
            average_layer = Lambda(function=lambda x: K.mean(x, axis=1))
            model.add(average_layer)

            self.model = model

In [32]:
# setup optimizer: ADAM algorithm
optimizer = Adam(learning_rate=1e-6)
# metrics for judging performance of model
metrics = ['categorical_accuracy']



dense_units = 128
hidden_units = 128
reg = 0.1
#lstm model
init = LSTM_model(hidden_units=hidden_units, dense_units=dense_units, reg=reg, dropout_rate=0.3, seq_length=16, num_classes=num_classes)
model = init.model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=metrics)

In [33]:
history = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=200,
    validation_data=(X_val, y_val),
)

Epoch 1/200
13/13 [==============================] - 3s 80ms/step - loss: 1.1164 - categorical_accuracy: 0.3662 - val_loss: 1.1218 - val_categorical_accuracy: 0.3125
Epoch 2/200
13/13 [==============================] - 0s 11ms/step - loss: 1.1160 - categorical_accuracy: 0.3636 - val_loss: 1.1185 - val_categorical_accuracy: 0.3125
Epoch 3/200
13/13 [==============================] - 0s 11ms/step - loss: 1.1095 - categorical_accuracy: 0.3485 - val_loss: 1.1156 - val_categorical_accuracy: 0.3125
Epoch 4/200
13/13 [==============================] - 0s 11ms/step - loss: 1.1120 - categorical_accuracy: 0.3914 - val_loss: 1.1128 - val_categorical_accuracy: 0.3125
Epoch 5/200
13/13 [==============================] - 0s 10ms/step - loss: 1.1057 - categorical_accuracy: 0.3889 - val_loss: 1.1102 - val_categorical_accuracy: 0.2917
Epoch 6/200
13/13 [==============================] - 0s 12ms/step - loss: 1.1071 - categorical_accuracy: 0.3763 - val_loss: 1.1079 - val_categorical_accuracy: 0.2917
Epoc

In [34]:
score = model.evaluate(x=X_train, y=y_train, verbose=1)
print("Train Loss without Classic Methods for all poses: %2.3f" % score[0])
print("Train Accuracy without Classic Methods for all poses: %1.3f\n" % score[1])

score = model.evaluate(x=X_val, y=y_val, verbose=1)
print("Val Loss without Classic Methods for all poses: %2.3f" % score[0])
print("Val Accuracy without Classic Methods for all poses: %1.3f\n" % score[1])

score = model.evaluate(x=X_test, y=y_test, verbose=1)
print("Test Loss without Classic Methods for all poses: %2.3f" % score[0])
print("Test Accuracy without Classic Methods for all poses: %1.3f\n" % score[1])

13/13 [==============================] - 0s 6ms/step - loss: 0.7861 - categorical_accuracy: 0.8056
Train Loss without Classic Methods for all poses: 0.786
Train Accuracy without Classic Methods for all poses: 0.806

2/2 [==============================] - 0s 12ms/step - loss: 0.8828 - categorical_accuracy: 0.6458
Val Loss without Classic Methods for all poses: 0.883
Val Accuracy without Classic Methods for all poses: 0.646

2/2 [==============================] - 0s 14ms/step - loss: 0.8302 - categorical_accuracy: 0.7647
Test Loss without Classic Methods for all poses: 0.830
Test Accuracy without Classic Methods for all poses: 0.765

